# Data Mining ⛏

**Purpose:** Collect all relevant Tweet's pertaining to the reopening of schools in the COVID-19 pandemic between Jan. 1, 2020 and Sept. 15, 2020.

**Pipeline:**
1.  Connect to Twitter's Search Tweets API (v2), to the `full archive` endpoint using the academic research track
2. *for each rumour* 
    1. Collect all tweets that contain a link to the rumour fact-checking website 
3. *for each of the linking tweets*
    1. Take that tweets conversation ID
    2. Collect the tweets entire thread by matching the conversation ID
    3. Store collection of tweets in Pandas dataframe, and only keep relevant features (text, author, *etc.*)
    4. Add an extra column that is the cleaned tweet text.
    5. Save dataframe to CSV
    6. Solve the pandemic

In [19]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm 
from searchtweets import collect_results, gen_request_parameters, load_credentials, ResultStream

search_args = load_credentials(filename="./api-secret.yaml",yaml_key="search_tweets_v2",env_overwrite=False)

In [2]:
query = gen_request_parameters("from:twitterdev")
print(query)

{"query": "from:twitterdev"}


In [3]:
tweets = collect_results(query,
                         max_tweets=10,
                         result_stream_args=search_args)

In [20]:
for tweet in tweets:
    if "text" in tweet: print(tweet["text"])

Introducing the manage Likes endpoints to the #TwitterAPI v2. Liking Tweets is one of the core aspects of the Twitter experience. Developers can now use these endpoints to like or unlike a specified Tweet on behalf of an authenticated account.

https://t.co/VCWYEHN8Bf
We will be starting soon. 

Join us on https://t.co/SQziibOD9P https://t.co/kGGfrrh52a
Join us next Thursday, April 22nd at 10 am PT for our next livestream 🚨

Getting started with the #TwitterAPI v2 and the new Academic Research product track using the twarc2 library in Python

We will be live on Twitch https://t.co/SQziibOD9P
Introducing the manage blocks endpoints to the #TwitterAPI v2. These endpoints are crucial for developers who want to help maintain and improve the health of the public conversation on Twitter.  Learn more 
https://t.co/8ayivhifZE
Interested in how our engineers tackled the challenge? Get a peek under the hood over at the @TwitterEng  blog 👇

Link 🔗 https://t.co/KexZ69tsye
Over a year ago, we relea